In [1]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

import nltk
# nltk.download('words')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import wordnet as wn
en_words = set(nltk.corpus.words.words())

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def tokenize(text):
    filter_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.like_url:
            continue
        elif token.orth_.startswith('@'):
            continue
        else:
            token = token.lower_
            filter_tokens.append(token)
    return filter_tokens

en_stop = set(nltk.corpus.stopwords.words('english'))
en_words = set(nltk.corpus.words.words())

def prepare_text(text):
    tokens = tokenize(text)
    tokens = [get_lemma(token) for token in tokens]
    tokens = [token for token in tokens if token in en_words]
    return tokens



In [2]:
print(" " in en_words)

False


In [15]:
import json
with open("../local/dataset_final.json", "r") as json_file:
    data = json.load(json_file)

In [29]:
from tqdm import tqdm
from copy import deepcopy
word_map = {"SOS":0, "EOS":1, "*":2}
len_dict = {}
data_batch = {}
for category in data.keys():
    data_batch[category] = {}
    data_by_category = data[category]
    for user in tqdm(data_by_category.keys()):
        tokens = []
        len_tokens = 0
        for query in data_by_category[user]["query"]:
            query_token = prepare_text(query)
            tokens += deepcopy(["<SOS>"] + query_token + ["<EOS>"])
            len_tokens += len(query_token)
        if len_tokens >= 10 and len_tokens <= 50:
            data_batch[category][user] = deepcopy(tokens)


100%|██████████| 50000/50000 [00:07<00:00, 6470.82it/s]


1333


NameError: name 'random' is not defined

In [34]:
import random
random.seed(0)
data_sample = {}
for category in data_batch.keys():
    print(len(data_batch[category].keys()))
    data_sample[category] = []
    userID = random.sample(list(data_batch[category].keys()), 1000)
    for user in userID:
        for token in data_batch[category][user]:
            if token not in word_map:
                word_map[token] = word_map["*"]
                word_map["*"] += 1
        data_sample[category].append(deepcopy(data_batch[category][user]))
print(data_sample["0"][0])
print(len(word_map.keys()))

1333
3619
3197
['<SOS>', 'colon', 'cancer', 'symptom', '<EOS>', '<SOS>', 'symptom', 'of', 'colon', 'cancer', '<EOS>', '<SOS>', 'cancer', 'symptom', '<EOS>', '<SOS>', 'symptom', 'of', 'cancer', '<EOS>', '<SOS>', 'symptom', 'of', 'ovarian', 'cancer', '<EOS>']
5842
